In [1]:
import pandas as pd
import json
from functools import reduce

In [2]:
answer_data = pd.read_excel('MitAut_20231222_element-use.xlsx', index_col=0)
answer_data[:1]

,Participant #,MIT_Question_no,Item_1,Item_2,Goal,FeedBack_question_1,FeedBack_question_2,FeedBack_question_3,Time_left,Answer,both_items_included
0,WEB8c0e84-06d0-f0e-06a8-77141c7bc46,1-01,デジタルカメラ,塗り絵本,個人の創造性と表現を培う,Y,Y,Y,-1,塗り絵本で塗った絵をデジタルカメラで取り込み、さまざまな加工ができるアプリを介して絵を色々な...,3


In [3]:
criteria = {
    'elements_used': answer_data['both_items_included']==3,
    'answer_complete': (answer_data['FeedBack_question_3']=='Y') | (answer_data['Time_left']<0),
    'problem_understood': (answer_data['FeedBack_question_1']=='Y') & (answer_data['FeedBack_question_2']=='Y'),
}
criteria['all'] = reduce(lambda x,y: x&y, criteria.values())
filtered_answers = answer_data[criteria['all']]

In [4]:
for key, mask in criteria.items():
    print(key, mask.mean())

elements_used 0.9369747899159664
answer_complete 0.900775694893342
problem_understood 0.7889463477698772
all 0.6832579185520362


In [5]:
novelty_checks = pd.read_excel('Sanity_check_solutions_template.xlsx', 'Novelty', index_col=0)
novelty_checks[:1]

,Item1,Item2,Goal,Novel solution,NOT novel solutions,Unnamed: 6,Unnamed: 7
Q_no,,,,,,,
1-01,Digital Camera,Coloring Book,Foster personal creativity and expression,Develop a digital camera that converts photos ...,Take photos with a digital camera and use them...,ok,NaN


In [6]:
feasibility_checks = pd.read_excel('Sanity_check_solutions_template.xlsx', 'Feasibility', index_col=0)
feasibility_checks[:1]

,Item1,Item2,Goal,Feasible solution,NOT feasible solutions,Unnamed: 6,Unnamed: 7
Q_no,,,,,,,
1-01,Digital Camera,Coloring Book,Foster personal creativity and expression,Create a digital camera feature that converts ...,Invent a digital camera that captures the mood...,ok,NaN


In [7]:
thelist = []
for id in filtered_answers['MIT_Question_no'].unique():
    qrows = filtered_answers[filtered_answers['MIT_Question_no'] == id]
    
    # Compensate for goal heterogeneity
    goal_n = 0
    for agoal in qrows['Goal'].unique():
        n = (qrows['Goal']==agoal).sum()
        if n > goal_n:
            goal = agoal
            goal_n = n

    thedict = {
        'id': id,
        'item1': qrows['Item_1'].iloc[0],
        'item2': qrows['Item_2'].iloc[0],
        'goal': goal,
        'answers': qrows['Answer'].to_list(),
        'sanity_checks': {
            'novelty': {
                'winner': novelty_checks.loc[id, 'Novel solution'],
                'loser': novelty_checks.loc[id, 'NOT novel solutions']
            },
            'feasibility': {
                'winner': feasibility_checks.loc[id, 'Feasible solution'],
                'loser': feasibility_checks.loc[id, 'NOT feasible solutions']
            }
        }
    }
    thelist.append(thedict)

In [8]:
with open('MIT-inputs-to-elo.json', 'w', encoding='utf-8') as thefile:
    json.dump(thelist, thefile, ensure_ascii=False, indent=1)
    thefile.write('\n')